In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/heart-disease-uci/heart.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')

In [4]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [5]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

data = [
    ['a', 1, 2],
    ['b', 1, 1],
    ['b', 2, 2],
    [np.nan, np.nan, np.nan]
]

X = pd.DataFrame(data)
xt = DataFrameImputer().fit_transform(X)

print('before...')
print(X)
print('after...')
print(xt)

before...
     0    1    2
0    a  1.0  2.0
1    b  1.0  1.0
2    b  2.0  2.0
3  NaN  NaN  NaN
after...
   0         1         2
0  a  1.000000  2.000000
1  b  1.000000  1.000000
2  b  2.000000  2.000000
3  b  1.333333  1.666667


**As the coulmn cp (chest pain) has missing values, we need to impute the data.**

**The data is numeric and hence mean stratergy will be a suitable choice.**

In [6]:
from sklearn.preprocessing import Imputer
imput = Imputer(missing_values='NaN',strategy='mean')
df = list(imput.fit_transform(df))

for i in range(303):
    for j in range(14):
        df[i][j] = math.ceil(df[i][j])

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
df = pd.DataFrame(df)

In [8]:
df=df.rename(columns={0: 'age', 1:'sex', 2:'cp', 3:'trestbps',4: 'chol',5: 'fbs',6: 'restecg',7: 'thalach',8: 'exang',9: 'oldpeak',10: 'slope',11: 'ca',12: 'thal',13:'target'})

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
x = df.iloc[:,:-1]

In [11]:
y = df.iloc[:,-1]

In [12]:
# We need to find the most important features in the dataset
model= RandomForestClassifier(n_estimators=100,random_state=0)
model.fit(x,y)
pd.Series(model.feature_importances_,index=x.columns).sort_values(ascending=False)

cp          0.130101
thal        0.121490
ca          0.113053
thalach     0.112534
age         0.098113
chol        0.082806
trestbps    0.078676
oldpeak     0.068036
exang       0.064240
slope       0.061597
sex         0.037024
restecg     0.020578
fbs         0.011751
dtype: float64

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [14]:
# To simplify it, I will group the number0-2 together as disease positive, number 3 as disease negative
number=[0,1,2]
for col in df.itertuples():

    if col.cp in number:
        df['cp'].replace(to_replace=col.cp, value=1, inplace=True)

# Testing the accuracy when the top 8 features are used for fitting

In [15]:
df_top8 = df.loc[:,['cp','oldpeak','thal','ca','thalach','age','chol','trestbps','exang']]

In [16]:
x_train,x_test,y_train,y_test = train_test_split(df_top8,y,test_size=0.25,random_state=0)
clf = RandomForestClassifier()
clf.fit(x_train,y_train)
prediction = clf.predict(x_test)
accuracy = accuracy_score(prediction,y_test)
cm = confusion_matrix(prediction,y_test)
prfs = precision_recall_fscore_support(prediction,y_test)
print('Accuracy: ',accuracy)
print('\n')
print('Confusion Matrix: ',cm)
print('\n')
print('Precision: ', prfs[0])
print('Recall:    ', prfs[1])
print('Fscore:    ', prfs[2])
print('Support:   ', prfs[3])

Accuracy:  0.8157894736842105


Confusion Matrix:  [[27  8]
 [ 6 35]]


Precision:  [0.81818182 0.81395349]
Recall:     [0.77142857 0.85365854]
Fscore:     [0.79411765 0.83333333]
Support:    [35 41]


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [17]:
maxim = 0
n_estimators=0
max_depth=0
max_cm=0
max_prfs=0
max_features=0
for i in range(5,15):
    for j in range(5,15):
        for k in range(5,13):
            x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)
            clf = RandomForestClassifier(n_estimators=i,max_depth=j,max_features=k)
            clf.fit(x_train,y_train)
            prediction = clf.predict(x_test)
            accuracy = accuracy_score(prediction,y_test)
            cm = confusion_matrix(prediction,y_test)
            prfs = precision_recall_fscore_support(prediction,y_test)
            if accuracy > maxim:
                maxim = accuracy
                n_estimators = i
                max_depth = j
                max_features = k
                max_cm = cm
                max_prfs=prfs
                
print(str(i)+" "+str(j)+" "+str(k)+" "+str(maxim))
print('\n')
print('Confusion Matrix: ',cm)
print('\n')
print('Precision: ', prfs[0])
print('Recall:    ', prfs[1])
print('Fscore:    ', prfs[2])
print('Support:   ', prfs[3])

14 14 12 0.868421052631579


Confusion Matrix:  [[25 10]
 [ 8 33]]


Precision:  [0.75757576 0.76744186]
Recall:     [0.71428571 0.80487805]
Fscore:     [0.73529412 0.78571429]
Support:    [35 41]


# Let's test if standardization can improve the accuracy

In [18]:
from sklearn.preprocessing import StandardScaler

In [19]:
x = df.iloc[:,:-1]
x_std = StandardScaler().fit_transform(x)

In [20]:
maxim = 0
n_estimators=0
max_depth=0
max_features=0
for i in range(5,15):
    for j in range(5,15):
        for k in range(5,13):
            x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)
            clf = RandomForestClassifier(n_estimators=i,max_depth=j,max_features=k)
            clf.fit(x_train,y_train)
            prediction = clf.predict(x_test)
            accuracy = accuracy_score(prediction,y_test)
            if accuracy > maxim:
                maxim = accuracy
                n_estimators = i
                max_depth = j
                max_features = k
print(str(i)+" "+str(j)+" "+str(k)+" "+str(maxim))

14 14 12 0.8947368421052632
